In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None) # default='warn'

In [14]:
# Load combined data
df_raw = pd.read_csv('../data/combined.csv')
df = df_raw.copy()

# Load hero feature data
df_features = pd.read_csv('../data/features.csv')

In [33]:
len(df)

5600752

In [48]:
# All slots add up to 660
df['slot_total'] = df[[f'hero{i}_slot' for i in range(0,10)]].sum(axis=1)
filt_1 = (df['slot_total']==660).values
print(len(filt_1[filt_1==True]))
print(len(filt_1[filt_1==False]))

5600742
10


In [49]:
# Valid hero ids (from exploratory, we know only invalid choice is 0)
filt_2 = [True for i in range(0,len(df))]

for i in range(0,10):
    filt_2 = filt_2 & (df[f'hero{i}_pick']!=0).values

print(len(filt_2[filt_2==True]))
print(len(filt_2[filt_2==False]))

5600731
21


In [52]:
# Match duration from 26-56 minutes
min_duration = 26*60 # minutes to seconds
max_duration = 56*60 # minutes to seconds

filt_3 = (df['duration']>=min_duration).values & (df['duration']<=max_duration).values

print(len(filt_3[filt_3==True]))
print(len(filt_3[filt_3==False]))

5078172
522580


In [57]:
filter = filt_1 & filt_2 & filt_3 # standard filter is all filters &'d
print(len(filter[filter==True]))
print(len(filter[filter==False]))

filter = pd.DataFrame(filter, columns=['filter']) # convert standard filter to df
filter.to_csv('../data/filter.csv', index=False)

5078155
522597
